# Hito 3: Corona virus (Covid-19)

## Presentación

Somos el equipo conformado por Raúl Cid, José Espina, Michelle Valenzuela y Alejandro Veragua. Nuestro *dataset* corresponde al [publicado en el portal Kaggle](https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset) construido y mantenido por la Universidad John Hopkins, al igual que para el hito 1

## Comentarios sobre el hito 1

Se realizó un análisis exploratorio, donde descubrimos varios datos inconsistentes y outliers
Se buscaron correlaciones contra atributos como el ingreso per cápita, la temperatura, el índice de pobreza de los diferentes países, pero sin éxito. Finalmente se realizó una prueba de concepto de clústering a partir de los coeficientes de un modelo de regeresión polinomial donde sí aparecieron resultados interesantes

## Metodología de trabajo para el hito 3

Considerando que los polinomios hacen "overfitting" cuando el grado es muy alto, aprovecharemos hacer algo similar a lo realizado en el hito 1, con la diferencia de usar un modelo auto-regresivo y hacer clustering sobre los coeficientes que resulten del modelo. En particular, quisimos experimentar con un modelo auto-regresivo con media móvil (ARMA). Como se tienen datos sólo de los últimos meses y el virus es nuevo, los datos no son estacionarios (o no hay suficientes datos para demostrar que lo son), por lo que usaremos una variante llamada ARIMA (donde la "i" hace referencia a la integración), que permite encontrar las diferencias no-estacionarias para lograr la estacionariedad (https://www.statisticshowto.com/arma-model/)

Además, se refuerza la idea de 2 artículos publicados que usan el mismo acercamiento
1) Benvenuto, D., Giovanetti, M., Vassallo, L., Angeletti, S., & Ciccozzi, M. (2020). Application of the ARIMA model on the COVID-2019 epidemic dataset. Data in brief, 105340.
2) Yang, Q., Wang, J., Ma, H., & Wang, X. (2020). Research on COVID-19 Based on ARIMA ModelΔ—Taking Hubei, China as an example to see the epidemic in Italy. Journal of Infection and Public Health.

NOTA IMPORTANTE: Para ejecutar este notebook, se necesitan scikit, matplotlib, numpy y statsmodels. Éste último para poder aplicar el modelo ARIMA. Se debe instalar con pip en el ambiente del libro:

$ pip install statsmodels

o con conda:

$ conda install -c conda-forge statsmodels

(https://www.statsmodels.org/stable/install.html)

NOTA IMPORTANTE 2: Calcular el modelo ARIMA toma un tiempo de cómputo importante

En detalle, el plan es:

* Calcular ARIMA para todos los países


## Configuración base del notebook
El bloque a continuación configura el libro, definiendo funciones y cargando las fuentes de datos

In [1]:
# Librería de álgebra lineal
import numpy as np 
# Procesamiento de datos y carga de archivos CSV
import pandas as pd
# Librearía gráfica
import matplotlib.pylab as plt
# Librería para usar recursos del sistema operativo
import os 
# Librearía usada para extraer el nombre del archivo de cada path
import ntpath
# Librería gráfica
import seaborn as sns
# Librearía para usar herramientas relacionadas con
# aprendizaje máquina. Se usó para preprocesar y
# calcular regresión
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
# Colores
from matplotlib import colors
# Ticker
import matplotlib.ticker as ticker
# Animation
import matplotlib.animation as animation
# Arima
from statsmodels.tsa.arima_model import ARIMA
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
# Configuración general
%load_ext autoreload
%autoreload 2

# Matplotlib inline
%matplotlib notebook
%matplotlib inline

# Se prepara carga de los archivos del dataset
paths = []
for dirname, _, filenames in os.walk('./novel-corona-virus-2019-dataset'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        paths.append(path);
dataFrames = {}
for path in paths:
    dataFrames[ntpath.basename(path)] = pd.read_csv(path)

## Modelo ARIMA
Usamos un modelo ARIMA p=14,d=1,q=0. El detalle de estos parámetros se explica a continuación (basado en https://machinelearningmastery.com/arima-for-time-series-forecasting-with-python/)

* El parámetro "p" es el número de observaciones pasadas que considera el modelo. También se le llama "orden". Usamos  14 debido a que es el tiempo que demora una persona es saber que tiene el virus, considerando tiempos de síntomas y resultado del exámen
* El parámetro "d" es el grado de diferenciación. Esto se usa para generar estacionariedad en series no estacionarias
* El parámetro "q" es el tamaño de la media móvil. Al escoger 0 los datos no se suavizan



In [9]:
dset_base = pd.read_csv('dataset.csv',)
dset_extra = pd.read_csv('atributos_extra/atributos_paises.csv')
dset = dset_base.join(dset_extra.set_index('pais'), on='pais')
pais_arima_coef = dict()
paises_error = []
for pais in dset_base.pais.unique() :
    dbset_actual = dset_base[dset_base['pais']==pais][['num_dia_desde_primer_caso','num_confirmados']]
    dbset_actual= dbset_actual.reset_index()
    series_actual = pd.Series(dbset_actual.num_confirmados.values, index=dbset_actual.num_dia_desde_primer_caso)
    try :
        model = ARIMA(series_actual.astype(float), order=(14,1,0),missing="drop")
        model_fit = model.fit(disp=False)
        pais_arima_coef[pais] = model_fit.params
    except :
        paises_error.append(pais)

# acá queda el diccionario con los coeficientes
print(pais_arima_coef)
print(paises_error)

C:\devel\conda\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\devel\conda\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\devel\conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\devel\conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\devel\conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. fore

{'Japan': const         3.300935
ar.L1.D.y    -0.834871
ar.L2.D.y    -0.628382
ar.L3.D.y    -0.580682
ar.L4.D.y    -0.350744
ar.L5.D.y    -0.276227
ar.L6.D.y     0.142869
ar.L7.D.y     0.302037
ar.L8.D.y     0.414380
ar.L9.D.y     0.346795
ar.L10.D.y    0.346406
ar.L11.D.y    0.226874
ar.L12.D.y   -0.043191
ar.L13.D.y    0.005405
ar.L14.D.y   -0.059558
dtype: float64, 'Mainland China': const        -0.292956
ar.L1.D.y    -0.614551
ar.L2.D.y    -0.569744
ar.L3.D.y    -0.413046
ar.L4.D.y    -0.279874
ar.L5.D.y    -0.195462
ar.L6.D.y    -0.171729
ar.L7.D.y    -0.124197
ar.L8.D.y    -0.072063
ar.L9.D.y     0.074632
ar.L10.D.y   -0.019744
ar.L11.D.y    0.158102
ar.L12.D.y    0.117405
ar.L13.D.y    0.064628
ar.L14.D.y    0.098612
dtype: float64, 'Thailand': const        -0.006564
ar.L1.D.y    -0.401141
ar.L2.D.y    -0.260290
ar.L3.D.y     0.005877
ar.L4.D.y    -0.020897
ar.L5.D.y     0.000790
ar.L6.D.y     0.079691
ar.L7.D.y     0.215984
ar.L8.D.y     0.148586
ar.L9.D.y     0.054731
ar.L10.D